# $\S$ 4.4.1. Fitting Logistic Regression Models

### Maximum likelihood

Logistic regression models are usually fit by maximum likelihood, using the conditional likelihood of $G$ given $X$. Since $\text{Pr}(G|X)$ completely specifies the conditional distribution, the *multinomial* distribution is appropriate.

The log-likelihood for $N$ observation is

\begin{equation}
l(\theta) = \sum_{i=1}^N \log p_{g_i}(x_i;\theta),
\end{equation}

where $p_k(x_i;\theta) = \text{Pr}(G=k|X=x_i;\theta)$

### Maximum likelihood for $K=2$ case

We discuss in detail the two-class case, sine the algorithms simplify considerably. It is convenient to code the two-class $g_i$ via a $0/1$ response $y_i$, where $y_i=1$ when $g_i=1$, and $0$ otherwise. Then we can let

\begin{align}
p_1(x;\theta) &= p(x;\theta), \\
p_2(x;\theta) &= 1- p(x;\theta). \\
\end{align}

The log-likelihood can be written

\begin{align}
l(\beta) &= \sum_{i=1}^N \left\lbrace y_i\log p(x_i;\beta) + (1-y_i)\log(1-p(x_i;\beta)) \right\rbrace \\
&= \sum_{i=1}^N \left\lbrace y_i\beta^Tx_i - \log(1+\exp(\beta^Tx)) \right\rbrace,
\end{align}

where $\beta^T = \lbrace \beta_{10}, \beta_1^T \rbrace$, and we assume that the vector of inputs $x_i$ includes the constant term 1 to acommodate the intercept.

To maximize the log-likelihood, we set its derivatives to zero. These *score* equations are

\begin{equation}
\frac{\partial l(\beta)}{\partial\beta} = \sum_{i=1}^N x_i(y_i-p(x_i;\beta)) = 0,
\end{equation}

which are $p+1$ equations *nonlinear* in $\beta$. Notice that since $x_{i1} =1$, the first score equation specifies that

\begin{equation}
\sum_{i=1}^N y_i = \sum_{i=1}^N p(x_i;\beta),
\end{equation}

implying that the *expected* number of class ones matches the observed number (and hence also class twos).

### Newton-Raphson algorithm

To solve the score equation, we use the Newton-Raphson algorithm, which requires the second-derivative or Hessian matrix

\begin{equation}
\frac{\partial^2 l(\beta)}{\partial\beta\partial\beta^T} = -\sum_{i=1}^N x_ix_i^T p(x_i;\beta)(1-p(x_i;\beta)).
\end{equation}

Starting with $\beta^{\text{old}}$, a single Newton update is

\begin{equation}
\beta^{\text{new}} = \beta^{\text{old}} - \left( \frac{\partial^2 l(\beta)}{\partial\beta\partial\beta^T} \right)^{-1} \frac{\partial l(\beta)}{\partial\beta},
\end{equation}

where the derivatives are evaluated at $\beta^{\text{old}}$.

### The same thing in matrix notation

Let
* $\mathbf{y}$ be the vector of $y_i$ values,
* $\mathbf{X}$ the $N\times (p+1)$ matrix of $x_i$ values,
* $\mathbf{p}$ the vector of fitted probabilities with $i$th element $p(x_i;\beta^{\text{old}})$, and
* $\mathbf{W}$ $N\times N$ diagonal matrix of weights with $i$th diagonal elements $p(x_i;\beta^{\text{old}})(1-p(x_i;\beta^{\text{old}}))$.

Then we have

\begin{align}
\frac{\partial l(\beta)}{\partial\beta} &= \mathbf{X}^T(\mathbf{y}-\mathbf{p}) \\
\frac{\partial^2l(\beta)}{\partial\beta\partial\beta^T} &= -\mathbf{X}^T\mathbf{WX},
\end{align}

and thus the Newton step is

\begin{align}
\beta^{\text{new}} &= \beta^{\text{old}} + (\mathbf{X}^T\mathbf{WX})^{-1}\mathbf{X}^T(\mathbf{y}-\mathbf{p}) \\
&= (\mathbf{X}^T\mathbf{WX})^{-1} \mathbf{X}^T\mathbf{W}\left( \mathbf{X}\beta^{\text{old}} + \mathbf{W}^{-1}(\mathbf{y}-\mathbf{p}) \right) \\
&= (\mathbf{X}^T\mathbf{WX})^{-1}\mathbf{X}^T\mathbf{W}\mathbf{z},
\end{align}

where we have re-expressed the Newton step as weighted least squares step, with the response

\begin{equation}
\mathbf{z} = \mathbf{X}\beta^{\text{old}} + \mathbf{W}^{-1}(\mathbf{y}-\mathbf{p}),
\end{equation}

sometimes known as the *adjusted response*.

### Iteratively reweighted least squares

These equations for the Newton step get solved repeatedly, since at each iteration $p$ changes, and hence so does $\mathbf{W}$ and $\mathbf{z}$. This algorithm is referred to as *iteratively reweighted least squares* or IRLS, since each iteration solves the weighted least squares problem:

\begin{equation}
\beta^{\text{new}} \leftarrow \arg\min_\beta (\mathbf{z}-\mathbf{X}\beta)^T\mathbf{W}(\mathbf{z}-\mathbf{X}\beta)
\end{equation}

It seems that $\beta=0$ is a good starting value, although convergence is never guaranteed. Typically the algorithm does converge, since the log-likelihood is concave, but overshooting can occur. In the rare cases that the log-likelihood decreases, step size halving will guarantee convergence.

### Multiclass case with $K\ge 3$

The Newton step can also be expressed as an IRLS, but with a *vector* of $K-1$ responses and a nondiagonal weight matrix per observation. (Exercise 4.4)

Alternatively coordinate-descent methods ($\S$ 3.8.6) can be used to maximize the log-likelihood efficiently.

The $\textsf{R}$ package $\textsf{glmnet}$ (Friedman et al., 2010) can fit very large logistic regression problems efficiently, both in $N$ and $p$.

### Goal of logistic regression

Logistic regression models are used mostly as a data analysis and inference tool, where the goal is to understand the role of the input variables in *explaning* the outcome. Typically many models are fit in a search for a parsimonious model involving a subset of the variables, possibly with some interactions terms. The following example illustrates some of the issues involved.